In [30]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

eps=1e-8

In [8]:
from numpy.linalg import eigvals as eig


def train_alternating(x_train,y_train,task_indexes,gamma,
                      D_ini,max_iter,method,kernel_method,f_method,Dmin_method):

    n_samples=len(x_train)
    T=len(task_indexes)


    if (max(max(abs(Dini-Dini.T))) > eps):
        print('D should be symmetric')
        return
    if (min(eig(Dini)) < -eps):
        print('D should be positive semidefinite')
        return
    if (abs(np.trace(Dini)-1) > 100*eps):
        print('D should have trace  1')
        return

    D = Dini
    if method=='feat':
        cost=[]
        U,S,V=numpy.linalg.svd(D)
        fS=f_method(diag(S))
        temp=np.sqrt(fS)
        tempi=np.where(temp>eps)
        temp[tempi]=1/temp[tempi]
        fD_isqrt=np.dot(np.dot(U,np.diag(temp)),U.T)
        
        for i in range(max_iter):
            new_trainx=np.dot(fD_isqrt,trainx)
            W,costf,err,reg=train_kernel(new_trainx,y_train,task_indexes,
                                        gamma,kernel_method)
            W=np.dot(fD_isqrt,W)
            costfunc.append([i,costf,err,reg])
            
            U,S,V=np.linalg.svd(D)
            if (dim>T):
                S.append(np.zeros((dim,dim-T)))
            Smin=Dmin_method(diag(S))
            D=np.dot(np.dot(U,np.diag(Smin)),U.T)
            
            fS=f_method(Smin)
            temp=sqrt(fS)
            tempi=np.where(temp>eps)
            temp[tempi]=1/temp[tempi]
            fD_isqrt=np.dot(np.dot(U,np.diag(temp)),U.T)
    
    if method=='independent':
        W,costfunc,err,reg=train_kernel(x_train,y_train,task_indexes,
                                       gamma,kernel_method)
        D=[]
    if method=='diagonal':
        if(np.linalg.norm(D-np.diag(np.diag(D)))>eps):
            print('D should be diagonal')
            return
    costfunc=[]
    
    fS=f_method(diag(S))
    temp=np.sqrt(fS)
    tempi=np.where(temp>eps)
    temp[tempi]=1/temp[tempi]
    fD_isqrt=np.diag(temp)
    
    for i in range(max_iter):
            new_trainx=np.dot(fD_isqrt,trainx)
            W,costf,err,reg=train_kernel(new_trainx,y_train,task_indexes,
                                        gamma,kernel_method)
            W=np.dot(fD_isqrt,W)
            costfunc.append([i,costf,err,reg])
            
            Smin=Dmin_method(np.sqrt(np.sum(W**2,axis=1)))
            D=np.diag(Smin)
            
            fS=f_method(Smin)
            temp=sqrt(fS)
            tempi=np.where(temp>eps)
            temp[tempi]=1/temp[tempi]
            fD_isqrt=np.diag(temp)
    return W,D,costfunc

In [ ]:
def train_kernel(x_train,y_train,task_indexes,kernel_method):
    num_data = np.shape(x_train)[1];
    dim = len(x_train);
    T = len(task_indexes);
    task_indexes[T+1] = num_data+1;

    costfunc = 0;
    err = 0;
    reg = 0;

    for t = 1:T
        x = x_train[:, task_indexes[t]:task_indexes[t+1]-1];
        y = y_train[:, task_indexes[t]:task_indexes[t+1]-1];
        K = np.dot(x.T,x);
        [a, costfunct, errt, regt] = kernel_method(K,y,gamma);
        W(:,t) = np.dot(x,a);

        costfunc = costfunc + costfunct;
        err = err + errt;
        reg = reg + regt;
    return W,costfunc,err,reg

In [27]:
def train_alternating_epsilon(trainx,trainy,task_indexes,gamma,Dini,
                              iterations,method,kernel_method,
                              f_method,Dmin_method,epsilon_init):

    if (epsilon_init < eps):
        W,D,costfunc = train_alternating(trainx,trainy,task_indexes,gamma,
                                         Dini,iterations,method,
                                         kernel_method,f_method,Dmin_method);
        mineps = 0;
        return;

    mincost = inf;
    epsilon = epsilon_init;

    i = 1;
    while (epsilon > eps):
        Dmin_e_method = lambda b: Dmin_method(sqrt(b**2+epsilon));
        
        [We,De,costfunc_e] = train_alternating(trainx,trainy,task_indexes,gamma,Dini,iterations,
            method,kernel_method,f_method,Dmin_e_method);
        s = np.linalg.svd(De);
        costfunc_e[:,1]= costfunc_e[:,1] + gamma * epsilon * sum(f_method(s));

        curcost = costfunc_e[len(costfunc_e)-1,1];
        if (curcost < mincost):
            mincost = curcost;
            mineps = epsilon;
            W = We;
            D = De;

        costfunc[i] = costfunc_e;
        i = i+1;
        epsilon = epsilon / 10;
    return W,D,costfunc,mineps

In [29]:
def test_error_unbalanced(W,testx,testy,task_indexes):

    T = len(task_indexes);
    testerrs = np.zeros((T,1));
    task_indexes[T+1] = len(testy)+1;
    dim = len(testx);

    for t in range(T):
        t_testx = testx[:, task_indexes(t):task_indexes(t+1)-1 ];
        t_testy = testy[task_indexes(t):task_indexes(t+1)-1].T;
        prediction = np.dot(W[:,t].T,t_testx);
        testerrs[t] = np.dot((t_testy - prediction),(t_testy - prediction).T) / (task_indexes(t+1)-task_indexes(t));
    return testerrs



In [36]:
def vec_inv(d):
    v = np.zeros(length(d),1);
    ind = np.where(d > eps);
    v[ind]= 1 / d[ind];
    return v


def run_code_example(gammas,trainx,trainy,testx,testy,task_indexes,task_indexes_test,cv_size,Dini,iterations,
    method, epsilon_init, fname):

    f_method= lambda b: b/sum(b)
    

    [theW,theD,costs,mineps] = train_alternating_epsilon(trainx, trainy, task_indexes, gammas, Dini, iterations, 
        method, 'kernel_regression', vec_inv, f_method, epsilon_init);

    testerrs = mean(test_error_unbalanced(theW,testx,testy,task_indexes_test));

    #save(sprintf('results_%s_%s_lin.mat',fname,method_str),'gammas','Dini','method_str',
    #    'testerrs','theW','theD','costs','mineps');
    return bestcv,bestgamma,cverrs,testerrs,theW,theD

In [60]:
import hdf5storage
f=hdf5storage.loadmat('school_1_indexes.mat')
data = f.get('tr') 
data = np.array(data) # For converting to numpy array

In [64]:
f

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Mon May 28 13:36:16 2007',
 '__version__': '1.0',
 'tr': array([[   24,    92,   155, ..., 15362, 15342, 15349]], dtype=uint16),
 'tr_indexes': array([[    1],
        [  151],
        [  219],
        [  290],
        [  449],
        [  479],
        [  521],
        [  556],
        [  640],
        [  760],
        [  896],
        [ 1057],
        [ 1141],
        [ 1159],
        [ 1184],
        [ 1266],
        [ 1358],
        [ 1435],
        [ 1571],
        [ 1651],
        [ 1753],
        [ 1817],
        [ 1850],
        [ 1959],
        [ 2109],
        [ 2136],
        [ 2186],
        [ 2295],
        [ 2346],
        [ 2448],
        [ 2636],
        [ 2705],
        [ 2846],
        [ 2981],
        [ 3058],
        [ 3181],
        [ 3270],
        [ 3378],
        [ 3432],
        [ 3607],
        [ 3685],
        [ 3790],
        [ 3865],
        [ 3961],
        [ 4066],
   